<a href="https://colab.research.google.com/github/Interactions-SpoofProof/interactions-ai-studio-project/blob/main/mean_pooling_train_dataset_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Steps for preparing the feature vectors for the neural network:
1. Use squeeze function to remove the 1 from the dimension (1, num_frames, 768)
2. Use mean pooling to reduce the vector to size (768)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gzip
import pickle

file_path = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/train_dataset_feature_vectors_01.pkl.gz'

with gzip.open(file_path, 'rb') as f:
     train_feature_vecs1 = pickle.load(f)

In [ ]:
file_path = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/train_dataset_feature_vectors_01.pkl.gz'

with gzip.open(file_path, 'rb') as f:
    train_feature_vecs2 = pickle.load(f)

In [ ]:
train_feature_vecs1[0].shape

(1, 220, 768)

In [ ]:
train_feature_vecs = train_feature_vecs1 + train_feature_vecs2

In [ ]:
len(train_feature_vecs)

8000

In [ ]:
import numpy as np

features_squeezed = [np.squeeze(feature_vec, axis=0) for feature_vec in train_feature_vecs]
pooled_feature_vecs = [np.mean(feature_vec, axis=0) for feature_vec in features_squeezed]

pooled_features = np.stack(pooled_feature_vecs)
print(pooled_features.shape)

Run the cells below for neural network training:

In [ ]:
import gzip
import pickle

with gzip.open('/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/train_labels.pkl.gz', 'rb') as f:
    labels = pickle.load(f) # contains just labels column

In [ ]:
import pandas as pd

train_df = pd.DataFrame(pooled_features)

print(len(labels))
train_df['label'] = labels[:]

8000


In [ ]:
#shuffling order of datapoints
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [ ]:
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Input, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [ ]:
import tensorflow as tf

# Check if GPU is available
print("GPU Available: ", tf.test.is_gpu_available())


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU Available:  False


In [ ]:
model = Sequential()

# f1: 0.7 - 256, 128 2layers
# f1: 0.94 - 256, 128, 64 3layers dropout=0.5 l2=0.01 adam=0.001

model.add(Input(shape=(X_train.shape[1],)))

# First hidden layer
model.add(Dense(256, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Second hidden layer
model.add(Dense(128, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(64, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))


# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile with lower learning rate
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


#history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


In [ ]:
# fit model to data (2 min) highest accuracy on train is 82%
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.7164 - loss: 5.2083 - val_accuracy: 0.7981 - val_loss: 2.0773
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7985 - loss: 1.6406 - val_accuracy: 0.7981 - val_loss: 0.9666
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7994 - loss: 0.8065 - val_accuracy: 0.7981 - val_loss: 0.6377
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8037 - loss: 0.5364 - val_accuracy: 0.7981 - val_loss: 0.5019
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7996 - loss: 0.4426 - val_accuracy: 0.7981 - val_loss: 0.5312
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8038 - loss: 0.4190 - val_accuracy: 0.7950 - val_loss: 0.4205
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8012 - loss: 0.4057 - val_accuracy: 0.7919 - val_loss: 0.5428
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8094 - loss: 0.4076 - val_accuracy: 

In [ ]:
file_path = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/test_dataset_feature_vectors.pkl.gz'

with gzip.open(file_path, 'rb') as f:
  test_feature_vecs = pickle.load(f)

In [ ]:
file_path = '/content/drive/MyDrive/Team Interactions: SpoofProof - AudioClassification/test_labels.pkl.gz'

with gzip.open(file_path, 'rb') as f:
  test_labels = pickle.load(f)

In [ ]:
import numpy as np

features_squeezed = [np.squeeze(feature_vec, axis=0) for feature_vec in test_feature_vecs]
pooled_feature_vecs = [np.mean(feature_vec, axis=0) for feature_vec in features_squeezed]

test_pooled_features = np.stack(pooled_feature_vecs)
print(test_pooled_features.shape)

(2000, 768)


In [ ]:
test_df = pd.DataFrame(test_pooled_features)

print(len(test_labels))
test_df['label'] = test_labels[:]

2000


In [ ]:
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [ ]:
X_test = test_df.drop('label', axis=1) # contains the feature vectors
y_test = test_df['label'] # contains just the labels

In [ ]:
# evaluating model
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {score[0]}, Test Accuracy: {score[1]}')


Test Loss: 0.24307668209075928, Test Accuracy: 0.9409999847412109


In [ ]:
# model makes preditions on test set
y_pred_prob = model.predict(X_test)

# Convert predicted probabilities to class labels (0 or 1)
y_pred = (y_pred_prob > 0.5).astype(int)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Calculate key metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
fpr = fp / (fp + tn)  # False positive rate
specificity = tn / (tn + fp)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall (Sensitivity): {recall}")
print(f"F1-Score: {f1}")
print(f"False Positive Rate: {fpr}")
print(f"Specificity: {specificity}")


Accuracy: 0.941
Precision: 1.0
Recall (Sensitivity): 0.705
F1-Score: 0.8269794721407625
False Positive Rate: 0.0
Specificity: 1.0


In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000, penalty='l2', C=1000)  # You can adjust max_iter for convergence

# Train the model
logistic_model.fit(X_train, y_train)

LogisticRegression(C=1000, max_iter=1000)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on test set
y_pred = logistic_model.predict(X_test)

# f1 = 0.6 for max_iter=1000
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}')

Accuracy: 0.8985, Precision: 1.0, Recall: 0.4925, F1-Score: 0.6599664991624791


In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier(loss="log_loss", max_iter=2000)
sgd_model.fit(X_train, y_train)

SGDClassifier(loss='log_loss', max_iter=2000)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on test set
y_pred = sgd_model.predict(X_test)

# Calculate accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}')


Accuracy: 0.9605, Precision: 1.0, Recall: 0.8025, F1-Score: 0.8904299583911235
